In [33]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from collections import defaultdict
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score,classification_report

## Problem 1
#### Building in house evaluaion metrics for our models such as Fscore, Precesion and Recall


In [2]:
## Calculates the precision of the predicted labels
def get_precision(y_pred, y_test1, debug = False):
    # deal with npdarray
    y_pred = list(y_pred)
    y_true = list(y_test1)
    #---------
    n = len(y_pred);
    y_pred = [0 if v is None else v for v in y_pred]# deal with None type
    y_true = [0 if v is None else v for v in y_test1]# deal with None type
    true_positive = sum(y_pred[i]* y_test1[i] for i in range(n))
    if (0 == sum(y_pred)): return 0
    return true_positive*1.0/sum(y_pred)
    
## Calculates the recall of the predicted labels
def get_recall(y_pred, y_test1):
    # deal with npdarray
    y_pred = list(y_pred)
    y_true = list(y_test1)
    #---------
    n = len(y_pred);
    y_pred = list(map(int,[1 == l for l in y_test1]))# deal with None type
    y_true = list(map(int,[1 == l for l in y_test1]))# deal with None type
    true_positive = sum(y_pred[i]*y_test1[i] for i in range(n))
    if 0 == sum(y_test): return 0
    return true_positive*1.0/sum(y_test1)
    

## Calculates the f-score of the predicted labels
def get_fscore(y_pred, y_test1):
    precision = get_precision(y_pred, y_test1);
    if (0 == precision): return 0
    recall= get_recall(y_pred, y_test1);
    if (0 == recall): return 0
    beta = 1.0;
    # print("get_fscore:",(beta**2*precision+recall))
    fscore = (beta**2+1)*precision*recall/(beta**2*precision+recall);
    return fscore

### Downloading feat files, transform and observe them

In [3]:
#import requests

#filename = 'aclImdb_v1.tar.gz'
#url = u'http://ai.stanford.edu/~amaas/data/sentiment/' + filename
#r = requests.get(url)
#with open(filename, 'wb') as f:
    #f.write(r.content)

# ...extract zip file
#import tarfile

#tar = tarfile.open(filename, mode='r')
#tar.extractall()
#tar.close()

In [4]:
#Load Files and divide them features and labels
import sklearn
from sklearn.datasets import load_svmlight_file
X_test, y_test = load_svmlight_file("Test_labeledBow.feat")
X_train, y_train = load_svmlight_file("Train_labeledBow.feat")
y_train

array([ 9.,  7.,  9., ...,  4.,  2.,  2.])

In [5]:
#Labels are given in review score turning them binary by given threshold
y_test= np.where(y_test <= 4, 0, 1)
y_train=np.where(y_train <= 4, 0, 1)
y_train

array([1, 1, 1, ..., 0, 0, 0])

In [47]:
#changing dtype for efficient memory usage 
import scipy.sparse._sparsetools
import scipy
X_test=scipy.sparse.lil_matrix(X_test, dtype='uint16')
X_train=scipy.sparse.lil_matrix(X_train, dtype='uint16')

In [57]:
#Changing to array as our test and train data in sparse matrix 
X_test_array=X_test.toarray()
X_train_array=X_train.toarray()

In [8]:
#Observing train data in dataframe
train_df=pd.DataFrame(X_train_array)
train_df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,89517,89518,89519,89520,89521,89522,89523,89524,89525,89526
0,9,1,4,4,6,4,2,2,4,0,...,0,0,0,0,0,0,0,0,0,0
1,7,4,2,2,0,4,1,0,2,2,...,0,0,0,0,0,0,0,0,0,0
2,4,4,4,7,2,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,10,2,2,0,3,2,4,2,0,1,...,0,0,0,0,0,0,0,0,0,0
4,13,9,6,4,2,5,10,6,0,2,...,0,0,0,0,0,0,0,0,0,0


In [10]:
#Observing test data in dataframe
test_df=pd.DataFrame(X_test_array)
test_df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,89513,89514,89515,89516,89517,89518,89519,89520,89521,89522
0,7,4,2,5,5,1,3,1,6,3,...,0,0,0,0,0,0,0,0,0,0
1,9,4,4,0,3,1,10,1,12,1,...,0,0,0,0,0,0,0,0,0,0
2,21,7,4,5,7,8,0,5,8,4,...,0,0,0,0,0,0,0,0,0,0
3,13,5,8,8,4,2,6,4,8,2,...,0,0,0,0,0,0,0,0,0,0
4,4,1,2,1,3,2,3,2,4,4,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# As you can see test data is less in columns thus we add default columns so that our test data will be suitable for model impl.
test_df['89523']='0'
test_df['89524']='0'
test_df['89525']='0'
test_df['89526']='0'
test_df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,89517,89518,89519,89520,89521,89522,89523,89524,89525,89526
0,7,4,2,5,5,1,3,1,6,3,...,0,0,0,0,0,0,0,0,0,0
1,9,4,4,0,3,1,10,1,12,1,...,0,0,0,0,0,0,0,0,0,0
2,21,7,4,5,7,8,0,5,8,4,...,0,0,0,0,0,0,0,0,0,0
3,13,5,8,8,4,2,6,4,8,2,...,0,0,0,0,0,0,0,0,0,0
4,4,1,2,1,3,2,3,2,4,4,...,0,0,0,0,0,0,0,0,0,0


## Problem 2 
### Majority Baseline


In [12]:
#in this question we will use Majority Baseline Model. 
#As data is even between neg and pos reviews we assign majority reviews as pos
#The label for the test data would be the majority class found in training data that is positive as we assigned
y_pred = [1]*len(test_df)
#Evaluating model performance
print('Fscore :', get_fscore(y_pred, y_test))
print('Recall', get_recall(y_pred, y_test))
print('Precesion',get_precision(y_pred, y_test, debug = False))


Fscore : 0.666666666667
Recall 1.0
Precesion 0.5


## Problem 3
### Length Baseline

#### Choosing threshold that maximizes accuracy

In [34]:
def review_length_threshold(df, label, plot_flag = True):
    ## YOUR CODE HERE
    # print("load training file")
    #words,labels = load_file(training_file, True)
    Dict_len = defaultdict(set)
    for w in df: Dict_len[w].add(w)
    # Evaluation depending on fscore
    Min = min(Dict_len)
    Max = max(Dict_len) 
    ThreRange=  range(Min,Max+1)
    FscoreL = dict([(f1_score(list(map(int,[w > Thres for w in df])), label, average="macro"),Thres) for Thres in ThreRange])
    Thres_opt = FscoreL[max(FscoreL)]
    if plot_flag:
         print("Range of thresholds:",Min,"to",Max, " with optimal threshold:", Thres_opt+1)
    del df
    del label
    return ThreRange, Thres_opt

In [38]:
train_df_length=train_df.sum(axis=1)
train_df_length.head(5)

0    138
1    114
2    115
3    104
4    232
dtype: int64

In [39]:
y_train

array([1, 1, 1, ..., 0, 0, 0])

In [40]:
review_length_threshold(train_df_length, y_train, plot_flag = True)

Range of thresholds: 10 to 2476  with optimal threshold: 194


/opt/anaconda/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


(range(10, 2477), 193)

#### Length for train

In [49]:
X_train_array=X_train_array.sum(axis=1)
X_train_array

array([138, 114, 115, ..., 231, 153, 138], dtype=uint64)

In [50]:
data2=np.where(X_train_array < 194, 0, 1)

In [51]:
data2

array([0, 0, 0, ..., 1, 0, 0])

In [52]:
print('Fscore :', get_fscore(data2, y_train))
print('Recall', get_recall(data2, y_train))
print('Precesion',get_precision(data2, y_train, debug = False))

Fscore : 0.671514852706
Recall 1.0
Precesion 0.505474113936


#### Length for test

In [58]:
X_test_array=X_test_array.sum(axis=1)
X_test_array

array([152, 141, 302, ..., 258, 167, 142], dtype=uint64)

In [59]:
data_test=np.where(X_test_array < 194, 0, 1)
data_test

array([0, 0, 1, ..., 1, 0, 0])

In [60]:
print('Fscore :', get_fscore(data_test, y_test))
print('Recall', get_recall(data_test, y_test))
print('Precesion',get_precision(data_test, y_test, debug = False))

Fscore : 0.659937888199
Recall 1.0
Precesion 0.492468134415


## Problem 4
### Naive Bayes Classifier

In [21]:
#initialize NB classifier
print("Naive-Bayes Classifier")
nb = GaussianNB()
#fit your classifier to training set
nb = nb.fit(train_df[:15000], y_train[:15000])


Naive-Bayes Classifier


In [22]:
y_pred1 = nb.predict(test_df[:15000])


In [24]:
print('Fscore :', get_fscore(y_pred1, y_test[:15000]))
print('Recall', get_recall(y_pred1, y_test[:15000]))
print('Precesion',get_precision(y_pred1, y_test[:15000], debug = False))

Fscore : 0.915222562012
Recall 1.0
Precesion 0.843696162882


In [26]:
# I want to check whether my evaluation metrics is true or not with build in function from sklearn
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score,classification_report
#compare y_pred with actual targets for  test set(y_test) and calculate precision, recall, f1-score
print("Precision: %0.2f" %precision_score(y_test[:15000], y_pred1 , average="macro"))
print("Recall:  %0.2f" %recall_score(y_test[:15000], y_pred1 , average="macro"))
print("F1-score:  %0.2f" %f1_score(y_test[:15000], y_pred1 , average="macro"))
print('Very good it is identical')

Precision: 0.53
Recall:  0.53
F1-score:  0.53
Very good it is identical


## Problem 5
### Decision Tree Classifier


In [13]:
#Decision Tree Classifier
print("Decision Tree Classifier")
dt = tree.DecisionTreeClassifier()
dt = dt.fit(train_df[:15000], y_train[:15000])

Decision Tree Classifier


In [14]:
y_pred = dt.predict(test_df[:15000])


In [15]:
print('Fscore :', get_fscore(y_pred, y_test[:15000]))
print('Recall', get_recall(y_pred, y_test[:15000]))
print('Precesion',get_precision(y_pred, y_test[:15000], debug = False))


Fscore : 1.0
Recall 1.0
Precesion 1.0


In [16]:
# I want to check whether my evaluation metrics is true or not with build in function from sklearn
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score,classification_report
#compare y_pred with actual targets for  test set(y_test) and calculate precision, recall, f1-score
print("Precision: %0.2f" %precision_score(y_test[:15000], y_pred , average="macro"))
print("Recall:  %0.2f" %recall_score(y_test[:15000], y_pred , average="macro"))
print("F1-score:  %0.2f" %f1_score(y_test[:15000], y_pred , average="macro"))
print('Very good it is identical')

Precision: 1.00
Recall:  1.00
F1-score:  1.00
Very good it is identical


# Part 2

In [3]:

import gensim 
from gensim.models import Word2Vec

ModuleNotFoundError: No module named 'gensim'

In [1]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install tensorflow-probability

  Using cached https://files.pythonhosted.org/packages/f8/72/29ef1e5f386b65544d4e7002dfeca1e55b099ed182cd6d405c21a19ae259/tensorflow_probability-0.8.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/24/fb/4f92f8c0f40a0d728b4f3d5ec5ff84353e705d8ff5e3e447620ea98b06bd/cloudpickle-1.1.1-py2.py3-none-any.whl
  Found existing installation: cloudpickle 1.2.1
    Uninstalling cloudpickle-1.2.1:


ERROR: spyder 3.3.6 requires pyqt5<5.13; python_version >= "3", which is not installed.
ERROR: spyder 3.3.6 requires pyqtwebengine<5.13; python_version >= "3", which is not installed.
ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\programdata\\anaconda3\\lib\\site-packages\\cloudpickle-1.2.1.dist-info\\INSTALLER'
Consider using the `--user` option or check the permissions.



In [4]:
 import tensorflow.python.ops.linalg

In [9]:
import datetime
import numpy as np
import os

import gensim
from gensim.test.utils import datapath, get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec

import tensorflow as tf
#from tensorflow import projector

print('gensim Version: %s' % (gensim.__version__))

class WordEmbedding:
    __author__ = "Edward Ma"
    __copyright__ = "Copyright 2018, Edward Ma"
    __credits__ = ["Edward Ma"]
    __license__ = "Apache"
    __version__ = "2.0"
    __maintainer__ = "Edward Ma"
    __email__ = "makcedward@gmail.com"

    def __init__(self, verbose=0):
        self.verbose = verbose
        
        self.model = {}
        
    def convert(self, source, ipnut_file_path, output_file_path):
        if source == 'glove':
            input_file = datapath(ipnut_file_path)
            output_file = get_tmpfile(output_file_path)
            glove2word2vec(input_file, output_file)
        elif source == 'word2vec':
            pass
        elif source == 'fasttext':
            pass
        else:
            raise ValueError('Possible value of source are glove, word2vec, fasttext')
        
    def load(self, source, file_path):
        print(datetime.datetime.now(), 'start: loading', source)
        if source == 'glove':
            self.model[source] = gensim.models.KeyedVectors.load_word2vec_format(file_path)
        elif source == 'word2vec':
            self.model[source] = gensim.models.KeyedVectors.load_word2vec_format(file_path, binary=True)
        elif source == 'fasttext':
            self.model[source] = gensim.models.wrappers.FastText.load_fasttext_format(file_path)
        else:
            raise ValueError('Possible value of source are glove, word2vec, fasttext')
            
        print(datetime.datetime.now(), 'end: loading', source)
            
        return self
    
    def get_model(self, source):
        if source not in ['glove', 'word2vec', 'fasttext']:
            raise ValueError('Possible value of source are glove, word2vec, fasttext')
            
        return self.model[source]
    
    def get_words(self, source, size=None):
        if source not in ['glove', 'word2vec', 'fasttext']:
            raise ValueError('Possible value of source are glove, word2vec, fasttext')
        
        if source in ['glove', 'word2vec']:
            if size is None:
                return [w for w in self.get_model(source=source).vocab]
            else:
                results = []
                for i, word in enumerate(self.get_model(source=source).vocab):
                    if i >= size:
                        break
                        
                    results.append(word)
                return results
            
        elif source in ['fasttext']:
            if size is None:
                return [w for w in self.get_model(source=source).wv.vocab]
            else:
                results = []
                for i, word in enumerate(self.get_model(source=source).wv.vocab):
                    if i >= size:
                        break
                        
                    results.append(word)
                return results
        
        return Exception('Unexpected flow')
    
    def get_dimension(self, source):
        if source not in ['glove', 'word2vec', 'fasttext']:
            raise ValueError('Possible value of source are glove, word2vec, fasttext')
        
        if source in ['glove', 'word2vec']:
            return self.get_model(source=source).vectors[0].shape[0]
            
        elif source in ['fasttext']:
            word = self.get_words(source=source, size=1)[0]
            return self.get_model(source=source).wv[word].shape[0]
        
        return Exception('Unexpected flow')
    
    def get_vectors(self, source, words=None):
        if source not in ['glove', 'word2vec', 'fasttext']:
            raise ValueError('Possible value of source are glove, word2vec, fasttext')
        
        if source in ['glove', 'word2vec', 'fasttext']:
            if words is None:
                words = self.get_words(source=source)
            
            embedding = np.empty((len(words), self.get_dimension(source=source)), dtype=np.float32)            
            for i, word in enumerate(words):
                embedding[i] = self.get_vector(source=source, word=word)
                
            return embedding
        
        return Exception('Unexpected flow')
    
    def get_vector(self, source, word, oov=None):
        if source not in ['glove', 'word2vec', 'fasttext']:
            raise ValueError('Possible value of source are glove, word2vec, fasttext')
            
        if source not in self.model:
            raise ValueError('Did not load %s model yet' % source)
        
        try:
            return self.model[source][word]
        except KeyError as e:
            raise
            
            #TODO
#             if oov is None:
#                 raise
            
#             if 'not in vocabulary' in str(e):
#                 if oov == ''

    def build_visual_metadata(self, embedding, words, file_dir, 
                              metadata_name='metadata.csv', project_model_name='model.ckpt'):
        # Create output directory if not exist
        if not os.path.exists(file_dir):
            os.makedirs(file_dir)

        # Build graph
        tf.reset_default_graph()
        sess = tf.InteractiveSession()

        embedding_graph = tf.Variable([0.0], name='embedding')
        place = tf.placeholder(tf.float32, shape=embedding.shape)

        set_embedding_graph = tf.assign(embedding_graph, place, validate_shape=False)
        sess.run(tf.global_variables_initializer())
        sess.run(set_embedding_graph, feed_dict={place: embedding})

        # Build metadata
        with open(os.path.join(file_dir, metadata_name), 'w') as f:
            for word in words:
                f.write(word + '\n')

        # Build projector
        summary_writer = tf.summary.FileWriter(file_dir, sess.graph)
        config = projector.ProjectorConfig()
        embedding_conf = config.embeddings.add()
        embedding_conf.tensor_name = 'embedding:0'
        embedding_conf.metadata_path = metadata_name
        projector.visualize_embeddings(summary_writer, config)

        # Save model
        saver = tf.train.Saver()
        saver.save(sess, os.path.join(file_dir, project_model_name))

        # Clear
        sess.close()

        
downloaded_glove_file_path = '../text/stanford/glove/glove.6B.50d.txt'
glove_file_path = '../text/stanford/glove/glove.840B.300d.vec'

word2vec_file_path = '../text/google/word2vec/GoogleNews-vectors-negative300.bin'
fasttext_file_path = '../text/facebook/fasttext/wiki.en.bin'

word_embedding = WordEmbedding

gensim Version: 3.8.1


In [12]:
word_embedding = WordEmbedding()
word_embedding.load(source='word2vec', file_path='../text/google/word2vec/GoogleNews-vectors-negative300.bin')
print(word_embedding.get_vector(source='word2vec', word='apple'))

2019-10-05 18:24:25.121007 start: loading word2vec


FileNotFoundError: [Errno 2] No such file or directory: '../text/google/word2vec/GoogleNews-vectors-negative300.bin'